In [ ]:
import zipfile
zip = zipfile.ZipFile(r"D:\study\Complete-Deep-Learning-Algorithms\Data\archive (1).zip",'r')
zip.extractall(r"D:\study\Complete-Deep-Learning-Algorithms\Data")
zip.close()

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
folder_path  =r'D:\study\Complete-Deep-Learning-Algorithms\Data\utkface_aligned_cropped\UTKFace'

age = []
gender = []
img_path =[]
for file in os.listdir(folder_path):

    age.append(int(file.split("_")[0]))
    gender.append(int(file.split("_")[1]))
    img_path.append(file)

In [3]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [4]:
df

,age,gender,img
0,100,0,100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,100_1_0_20170112213303693.jpg.chip.jpg
...,...,...,...
23703,9,1,9_1_3_20161220222856346.jpg.chip.jpg
23704,9,1,9_1_3_20170104222949455.jpg.chip.jpg
23705,9,1,9_1_4_20170103200637399.jpg.chip.jpg
23706,9,1,9_1_4_20170103200814791.jpg.chip.jpg


In [5]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=30,width_shift_range=0.2,
                                   height_shift_range = 0.2,
                                   shear_range=0.2,
                                   zoom_range =0.2,
                                   horizontal_flip=True)

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

Found 20000 validated image filenames.


In [9]:
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 3708 validated image filenames.


In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


## from ReseNet50 pretrained model , will take convolution base

# include_top will drop FC layer and only take convolution base 
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [11]:
# freeze trainable parameter of convolution layers
resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)


output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [12]:
# combine both output of outout1 and output2
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [24]:
# from the model we want to output one regression and second is classification 
# so that's why we given two loss function,metrics
# loss weight give the importanance for which prediction
# loss_weights={'age':1,'gender':99}
model.compile(optimizer='Adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'})

In [25]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

Found 20000 validated image filenames.
Found 3708 validated image filenames.


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.